## XGBoost

For detailed model descriptions and execution instructions, please refer to readme_xgboost.md.

In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
from google.colab import drive
drive.mount('/content/drive')
train_csv_path = "/content/drive/My Drive/CS412/train.csv"
df = pd.read_csv(train_csv_path)
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName                ExitStreetName EntryHeading  \
0  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
1  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
2  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   
3  Marietta Boulevard Northwest  Marietta Boulevard Northwest           SE   
4  Marietta Boulevard Northwest  Marietta Boulevard Northwest           NW   

  ExitHeading  Hour  Weekend  ...  TimeFromFirstStop_p40  \
0          NW     0        0 

In [12]:
df = pd.read_csv(train_csv_path)
x_train = df[['IntersectionId','EntryHeading','ExitHeading']]
x_train = pd.concat([x_train,pd.get_dummies(df["City"],dummy_na=False, drop_first=False, dtype=int)],axis=1)
print(x_train.head())
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
x_train['EntryHeading'] = x_train['EntryHeading'].replace(heading_map)
x_train['ExitHeading'] = x_train['ExitHeading'].replace(heading_map)
print(x_train)


   IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0               0           NW          NW        1       0        0   
1               0           SE          SE        1       0        0   
2               0           NW          NW        1       0        0   
3               0           SE          SE        1       0        0   
4               0           NW          NW        1       0        0   

   Philadelphia  
0             0  
1             0  
2             0  
3             0  
4             0  
        IntersectionId EntryHeading ExitHeading  Atlanta  Boston  Chicago  \
0                    0            8           8        1       0        0   
1                    0            4           4        1       0        0   
2                    0            8           8        1       0        0   
3                    0            4           4        1       0        0   
4                    0            8           8        1       0        0 

In [13]:
# Hour: 0-23 cyclical -> [sin(2pi*hour/24), cos(2pi*hour/24)]
df['hour_sin'] = np.sin(2*np.pi*df['Hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['Hour']/24)
print(df.head())
print(df['Month'].unique())

df['month_sin'] = np.sin(2*np.pi*df['Month']/12)
df['month_cos'] = np.cos(2*np.pi*df['Month']/12)
print(df.head())

     RowId  IntersectionId   Latitude  Longitude  \
0  1921357               0  33.791659 -84.430032   
1  1921358               0  33.791659 -84.430032   
2  1921359               0  33.791659 -84.430032   
3  1921360               0  33.791659 -84.430032   
4  1921361               0  33.791659 -84.430032   

                EntryStreetName  ... DistanceToFirstStop_p60  \
0  Marietta Boulevard Northwest  ...                     0.0   
1  Marietta Boulevard Northwest  ...                     0.0   
2  Marietta Boulevard Northwest  ...                     0.0   
3  Marietta Boulevard Northwest  ...                     0.0   
4  Marietta Boulevard Northwest  ...                     0.0   

  DistanceToFirstStop_p80     City  hour_sin  hour_cos  
0                     0.0  Atlanta  0.000000  1.000000  
1                     0.0  Atlanta  0.000000  1.000000  
2                     0.0  Atlanta  0.258819  0.965926  
3                     0.0  Atlanta  0.258819  0.965926  
4                

In [14]:
x_train['hour_sin'] = df['hour_sin']
x_train['hour_cos'] = df['hour_cos']
x_train['Hour'] = df['Hour']
# x_train['month_sin'] = df['month_sin']
# x_train['month_cos'] = df['month_cos']
x_train['Month'] = df['Month']
x_train['Weekend'] = df['Weekend']
print(x_train)

        IntersectionId EntryHeading ExitHeading  Atlanta  Boston  ...  \
0                    0            8           8        1       0  ...   
1                    0            4           4        1       0  ...   
2                    0            8           8        1       0  ...   
3                    0            4           4        1       0  ...   
4                    0            8           8        1       0  ...   
...                ...          ...         ...      ...     ...  ...   
856382            1990            5           7        0       0  ...   
856383            1990            5           5        0       0  ...   
856384            1990            5           5        0       0  ...   
856385            1990            7           7        0       0  ...   
856386            1990            5           5        0       0  ...   

        hour_sin      hour_cos  Hour  Month  Weekend  
0       0.000000  1.000000e+00     0      6        0  
1       0.000

In [15]:
y_train = pd.DataFrame()
y_train['TotalTimeStopped_p20'] = df['TotalTimeStopped_p20']
y_train['TotalTimeStopped_p50'] = df['TotalTimeStopped_p50']
y_train['TotalTimeStopped_p80'] = df['TotalTimeStopped_p80']
y_train['DistanceToFirstStop_p20'] = df['DistanceToFirstStop_p20']
y_train['DistanceToFirstStop_p50'] = df['DistanceToFirstStop_p50']
y_train['DistanceToFirstStop_p80'] = df['DistanceToFirstStop_p80']
print(y_train.head())

   TotalTimeStopped_p20  TotalTimeStopped_p50  TotalTimeStopped_p80  \
0                   0.0                   0.0                   0.0   
1                   0.0                   0.0                   0.0   
2                   0.0                   0.0                   0.0   
3                   0.0                   0.0                   0.0   
4                   0.0                   0.0                   0.0   

   DistanceToFirstStop_p20  DistanceToFirstStop_p50  DistanceToFirstStop_p80  
0                      0.0                      0.0                      0.0  
1                      0.0                      0.0                      0.0  
2                      0.0                      0.0                      0.0  
3                      0.0                      0.0                      0.0  
4                      0.0                      0.0                      0.0  


In [16]:
#Split data into train and val
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

Our Own Method-XGBoost

In [17]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [20]:
# Ensure numeric types for categorical features
x_train['EntryHeading'] = x_train['EntryHeading'].astype(int)
x_train['ExitHeading'] = x_train['ExitHeading'].astype(int)
x_val['EntryHeading'] = x_val['EntryHeading'].astype(int)
x_val['ExitHeading'] = x_val['ExitHeading'].astype(int)

from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor

scaler_y = StandardScaler()

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled = scaler_y.transform(y_val)

# confirm
print("y_train_scaled mean (should be ~0):", y_train_scaled.mean(axis=0))
print("y_train_scaled std (should be ~1):", y_train_scaled.std(axis=0))

base_model = xgb.XGBRegressor(
    objective='reg:squarederror', # regression
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist',
    eval_metric='rmse',
    random_state=42,
    device='cuda'
)

model = MultiOutputRegressor(base_model)

model.fit(x_train, y_train_scaled)

y_pred_scaled = model.predict(x_val)

y_pred = scaler_y.inverse_transform(y_pred_scaled)

rmse = np.sqrt(mean_squared_error(y_val,y_pred))
print('validation RMSE:',rmse)

rmses = np.sqrt(np.mean((y_val - y_pred)**2, axis=0))
print("Per-target RMSE:", rmses)

y_train_scaled mean (should be ~0): [ 3.58741065e-17 -8.05845064e-18  4.51356206e-17  8.48263420e-17
  6.01531708e-18 -1.78302295e-16]
y_train_scaled std (should be ~1): [1. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [00:58:16] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [00:58:16] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [01:00:25] WARNING: /workspace/src/context.cc:53: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [01:00:25] WARNING: /workspace/src/context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/

validation RMSE: 57.755715355711764
Per-target RMSE: TotalTimeStopped_p20         6.197466
TotalTimeStopped_p50        12.067221
TotalTimeStopped_p80        20.963552
DistanceToFirstStop_p20     28.738361
DistanceToFirstStop_p50     62.791642
DistanceToFirstStop_p80    120.922111
dtype: float64


In [21]:
import numpy as np
print("y_train mean:", np.mean(y_train))
print("y_train std:", np.std(y_train))
print("y_train min:", np.min(y_train))
print("y_train max:", np.max(y_train))


y_train mean: 25.300026857040265
y_train std: TotalTimeStopped_p20         7.145344
TotalTimeStopped_p50        15.681923
TotalTimeStopped_p80        28.282988
DistanceToFirstStop_p20     29.079167
DistanceToFirstStop_p50     74.878299
DistanceToFirstStop_p80    160.531433
dtype: float64
y_train min: 0.0
y_train max: 4079.2


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3800: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


## Load Test Data


In [22]:
import pandas as pd

test_csv_path = "/content/drive/My Drive/CS412/test.csv"
#test_csv_path = '/kaggle/input/bigquery-geotab-intersection-congestion/test.csv'
df_test = pd.read_csv(test_csv_path)
print(df_test.head())

   RowId  IntersectionId  Latitude  Longitude             EntryStreetName  \
0      0               1  33.75094 -84.393032  Peachtree Street Southwest   
1      1               1  33.75094 -84.393032  Peachtree Street Southwest   
2      2               1  33.75094 -84.393032  Peachtree Street Southwest   
3      3               1  33.75094 -84.393032  Peachtree Street Southwest   
4      4               1  33.75094 -84.393032  Peachtree Street Southwest   

               ExitStreetName EntryHeading ExitHeading  Hour  Weekend  Month  \
0  Peachtree Street Southwest           NE          NE     0        0      6   
1   Mitchell Street Southwest           SW          SE     0        0      6   
2  Peachtree Street Southwest           SW          SW     0        0      6   
3  Peachtree Street Southwest           NE          NE     1        0      6   
4  Peachtree Street Southwest           SW          SW     1        0      6   

                                                Path    

In [23]:
import pandas as pd
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

test_csv_path = "/content/drive/My Drive/CS412/test.csv"

# Load the dataset
df_test = pd.read_csv(test_csv_path)
print(df_test.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   RowId  IntersectionId  Latitude  Longitude             EntryStreetName  \
0      0               1  33.75094 -84.393032  Peachtree Street Southwest   
1      1               1  33.75094 -84.393032  Peachtree Street Southwest   
2      2               1  33.75094 -84.393032  Peachtree Street Southwest   
3      3               1  33.75094 -84.393032  Peachtree Street Southwest   
4      4               1  33.75094 -84.393032  Peachtree Street Southwest   

               ExitStreetName EntryHeading ExitHeading  Hour  Weekend  Month  \
0  Peachtree Street Southwest           NE          NE     0        0      6   
1   Mitchell Street Southwest           SW          SE     0        0      6   
2  Peachtree Street Southwest           SW          SW     0        0      6   
3  Peachtree Street Southwest           NE          NE     1        0      6   
4  Peach

## Preprocess Test Data

### Subtask:
Apply feature engineering transformations to the test dataset `df_test` to prepare it for prediction, ensuring it matches the training data structure.


In [24]:
import numpy as np
import pandas as pd

# Suppress FutureWarning by opting into future behavior
pd.set_option('future.no_silent_downcasting', True)

# 1. Map EntryHeading and ExitHeading
heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
# use infer_objects(copy=False) to handle future downcasting behavior explicitly
df_test['EntryHeading'] = df_test['EntryHeading'].replace(heading_map).infer_objects(copy=False)
df_test['ExitHeading'] = df_test['ExitHeading'].replace(heading_map).infer_objects(copy=False)

# 2. One-hot encode City
city_dummies = pd.get_dummies(df_test['City'], prefix='', prefix_sep='', dtype=int)
# Ensure all city columns exist
for city in ['Atlanta', 'Boston', 'Chicago', 'Philadelphia']:
    if city not in city_dummies.columns:
        city_dummies[city] = 0

# 3. Create cyclical features
df_test['hour_sin'] = np.sin(2 * np.pi * df_test['Hour'] / 24)
df_test['hour_cos'] = np.cos(2 * np.pi * df_test['Hour'] / 24)

# Concatenate dummies to df_test safely (check if columns exist to avoid duplicates on retry)
cols_to_add = [c for c in city_dummies.columns if c not in df_test.columns]
if cols_to_add:
    df_test = pd.concat([df_test, city_dummies[cols_to_add]], axis=1)

# 4. Select features
features = ['IntersectionId', 'EntryHeading', 'ExitHeading', 'Atlanta', 'Boston', 'Chicago', 'Philadelphia', 'hour_sin', 'hour_cos', 'Hour', 'Month', 'Weekend']
x_test = df_test[features]

print(x_test.head())

   IntersectionId  EntryHeading  ExitHeading  Atlanta  Boston  Chicago  \
0               1             2            2        1       0        0   
1               1             6            4        1       0        0   
2               1             6            6        1       0        0   
3               1             2            2        1       0        0   
4               1             6            6        1       0        0   

   Philadelphia  hour_sin  hour_cos  Hour  Month  Weekend  
0             0  0.000000  1.000000     0      6        0  
1             0  0.000000  1.000000     0      6        0  
2             0  0.000000  1.000000     0      6        0  
3             0  0.258819  0.965926     1      6        0  
4             0  0.258819  0.965926     1      6        0  


In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
import os
import numpy as np

# Define features and targets
features = ['IntersectionId', 'EntryHeading', 'ExitHeading', 'Atlanta', 'Boston', 'Chicago', 'Philadelphia', 'hour_sin', 'hour_cos', 'Hour', 'Month', 'Weekend']
target_cols = ['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']

# Conditional training if model is missing from environment
if 'model' not in locals() or 'scaler_y' not in locals():
    print("Model or scaler not found. Retraining model...")
    # Locate train.csv
    train_path = "/content/drive/MyDrive/CS412/train.csv"
    if not os.path.exists(train_path):
        train_path = "/content/drive/MyDrive/CS412/Stage1/train.csv"

    if os.path.exists(train_path):
        df_train = pd.read_csv(train_path)

        # Preprocess train
        heading_map = {'N':1,'NE':2,'E':3,'SE':4,'S':5, 'SW':6, 'W':7, 'NW': 8}
        # Use infer_objects to avoid future warning
        pd.set_option('future.no_silent_downcasting', True)
        df_train['EntryHeading'] = df_train['EntryHeading'].replace(heading_map).infer_objects(copy=False)
        df_train['ExitHeading'] = df_train['ExitHeading'].replace(heading_map).infer_objects(copy=False)

        city_dummies_train = pd.get_dummies(df_train['City'], prefix='', prefix_sep='', dtype=int)
        df_train = pd.concat([df_train, city_dummies_train], axis=1)

        df_train['hour_sin'] = np.sin(2 * np.pi * df_train['Hour'] / 24)
        df_train['hour_cos'] = np.cos(2 * np.pi * df_train['Hour'] / 24)

        x_train_ft = df_train[features]
        y_train_ft = df_train[target_cols]

        # Scale targets
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train_ft)

        # Train model (using reduced estimators for quick restoration if needed, adjust as preferred)
        base_model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100, # Set to 100 for quick fallback execution; original was 1000
            learning_rate=0.05,
            max_depth=10,
            subsample=0.8,
            colsample_bytree=0.8,
            tree_method='hist',
            random_state=42,
            n_jobs=-1
        )
        model = MultiOutputRegressor(base_model)
        model.fit(x_train_ft, y_train_scaled)
        print("Model retrained successfully.")
    else:
        raise FileNotFoundError("train.csv not found. Cannot train model.")

# Predict
print("Generating predictions...")
# Ensure x_test has correct column order
x_test_ordered = x_test[features]

y_pred_scaled = model.predict(x_test_ordered)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Create Submission DataFrame
submission = pd.DataFrame(y_pred, columns=target_cols)
submission['RowId'] = df_test['RowId']

# Reorder columns: RowId first
submission = submission[['RowId'] + target_cols]

# Save
submission.to_csv('submission.csv', index=False)
print("submission.csv saved.")
print(submission.head())

Generating predictions...
submission.csv saved.
   RowId  TotalTimeStopped_p20  TotalTimeStopped_p50  TotalTimeStopped_p80  \
0      0             -0.282285              2.692895             25.535791   
1      1              0.593424             10.517080             29.310316   
2      2              1.165542              3.271172              6.917352   
3      3              0.808450              3.384092             20.554977   
4      4              2.060323              3.452912              8.021889   

   DistanceToFirstStop_p20  DistanceToFirstStop_p50  DistanceToFirstStop_p80  
0                 3.217354                15.723953                39.152782  
1                -6.646869                24.011236                46.608898  
2                 2.317006                 1.224688                14.029455  
3                 4.159653                19.618063                37.147423  
4                 2.159608                 0.571972                20.467161  


In [30]:
import pandas as pd

# 1. Define the mapping of columns to IDs (0-5)
target_cols = [
    'TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80',
    'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80'
]
metric_map = {col: i for i, col in enumerate(target_cols)}

print("Converting to long format with mapping:", metric_map)

# 2. Melt the dataframe to long format
submission_long = submission.melt(id_vars=['RowId'], value_vars=target_cols, var_name='Metric', value_name='Target')

# 3. Create the 'ID' column (RowId_MetricId)
submission_long['MetricId'] = submission_long['Metric'].map(metric_map)
submission_long['TargetId'] = submission_long['RowId'].astype(str) + '_' + submission_long['MetricId'].astype(str)

# 4. Select final columns and sort
# Fix: Sort FIRST, then select columns so RowId is available for sorting
final_submission = submission_long.sort_values(by=['RowId', 'MetricId'])[['TargetId', 'Target']]

# 5. Save to file and Drive
final_submission.to_csv('submission_formatted.csv', index=False)
final_submission.to_csv('/content/drive/My Drive/CS412/submission_formatted.csv', index=False)

print("Formatted submission saved to 'submission_formatted.csv'.")
print(final_submission.head(10))

Converting to long format with mapping: {'TotalTimeStopped_p20': 0, 'TotalTimeStopped_p50': 1, 'TotalTimeStopped_p80': 2, 'DistanceToFirstStop_p20': 3, 'DistanceToFirstStop_p50': 4, 'DistanceToFirstStop_p80': 5}
Formatted submission saved to 'submission_formatted.csv'.
        TargetId     Target
0            0_0  -0.282285
1921357      0_1   2.692895
3842714      0_2  25.535791
5764071      0_3   3.217354
7685428      0_4  15.723953
9606785      0_5  39.152782
1            1_0   0.593424
1921358      1_1  10.517080
3842715      1_2  29.310316
5764072      1_3  -6.646869


In [36]:
import pandas as pd
import os

# Paths
data_dir = '/content/drive/My Drive/CS412/'
sample_sub_path = os.path.join(data_dir, 'sample_submission.csv')
formatted_sub_path = 'submission_formatted.csv'

# Check if sample_submission exists
if os.path.exists(sample_sub_path):
    print(f"Loading sample submission from {sample_sub_path}...")
    sample_sub = pd.read_csv(sample_sub_path)
    print("Sample submission shape:", sample_sub.shape)

    # Load our generated submission
    my_sub = pd.read_csv(formatted_sub_path)
    print("My submission shape (before fixing):", my_sub.shape)

    # Merge to keep only the IDs present in sample_submission
    # We use a left merge on sample_sub to enforce its order and rows
    final_sub_aligned = sample_sub[['TargetId']].merge(my_sub, on='TargetId', how='left')

    # Fill missing values if any (though ideally there shouldn't be any if my_sub is a superset)
    if final_sub_aligned['Target'].isnull().sum() > 0:
        print(f"Warning: {final_sub_aligned['Target'].isnull().sum()} missing predictions found. Filling with 0.")
        final_sub_aligned['Target'] = final_sub_aligned['Target'].fillna(0)

    # Save
    final_sub_aligned.to_csv('submission_fixed.csv', index=False)
    final_sub_aligned.to_csv(os.path.join(data_dir, 'submission_fixed.csv'), index=False)
    print("Fixed submission saved to 'submission_fixed.csv'.")
    print("New shape:", final_sub_aligned.shape)

else:
    print("sample_submission.csv not found in the CS412 folder. Please upload it or verify the path.")
    # Print current test shape to debug
    print("Current df_test shape:", df_test.shape)

Loading sample submission from /content/drive/My Drive/CS412/sample_submission.csv...
Sample submission shape: (11522010, 2)
My submission shape (before fixing): (11528142, 2)
Fixed submission saved to 'submission_fixed.csv'.
New shape: (11522010, 2)


In [37]:
import shutil
import os

# Define source and destination paths
source_path = 'submission.csv'
destination_folder = '/content/drive/My Drive/CS412/'
destination_path = os.path.join(destination_folder, 'submission_fixed.csv')

# Check if destination folder exists, if not create it (or just copy to root if preferred, but here we assume CS412 exists)
if not os.path.exists(destination_folder):
    print(f"Folder {destination_folder} not found. Saving to My Drive root instead.")
    destination_folder = '/content/drive/My Drive/'
    destination_path = os.path.join(destination_folder, 'submission_fixed.csv')

# Copy the file
shutil.copy(source_path, destination_path)
print(f"File saved successfully to: {destination_path}")

File saved successfully to: /content/drive/My Drive/CS412/submission_fixed.csv
